In [1]:
import pylab
import h5py
import math
import array
from numpy import *
import numpy as np
from pycbc.types import TimeSeries, FrequencySeries
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc import types, fft, waveform
import lal
from scipy import interpolate
from scipy.interpolate import interp1d
from lal import MSUN_SI, MTSUN_SI, G_SI, PC_SI, C_SI, PI
from pycbc.filter import match
from pycbc.psd import aLIGOZeroDetHighPower

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
#plt.rcParams["font.family"] = "monospace"
 
from matplotlib import gridspec

import matplotlib.pyplot as plt

## training set

In [2]:
tshift_byM = [344.99999999999994]
tmatch_byM = [-450.2451515296988] 
hyb = [1364]

In [3]:
#print(tshift_byM)
#print(hyb)
print(tshift_byM)

[344.99999999999994]


In [4]:
hfile = ['1364hyb_EccTD_xlow045_GM.h5']
e = [0.172]
q=2
Mtot=30

In [5]:
freqJoin = []
freqJoin_byM = []
freqm = []
freqidx = []
hyb = np.array(hyb)
for Id in range(0,len(hfile)):
    hyb_name = hfile[Id][0:4]
    hid = np.argmin(abs(hyb-float(hyb_name)))
    #f = h5py.File('/home/pratul/Downloads/Project/New hybrids/1364_EccTD_22_hyb_poster.h5','r')
    f = h5py.File('/home/pratul/Downloads/Project/New hybrids/TT2_Hybs/1364_TT2_Ebersold_22_hyb.h5','r')
    data = f['l2_m2']
    tVec = real(data[:,0])
    h22pVec = real(data[0:,1])
    h22cVec = real(data[0:,2])
    #print(hid)
    mode2polfac=4*(5/(64*np.pi))**(1/2)
    hpVec=h22pVec
    hcVec=h22cVec

    #tVec = tVec - tVec[np.argmax(abs(hpVec + 1j*hcVec))]

    M=Mtot
    M1 = q*M/(1+q)
    M2 = 1*M/(1+q)

    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    tVec=tVec*M*MTSUN_SI
    hpVec=hpVec*G_SI*M_SI/D_SI/C_SI/C_SI * mode2polfac
    hcVec=hcVec*G_SI*M_SI/D_SI/C_SI/C_SI * mode2polfac

    tVec = tVec - tVec[np.argmax(abs(hpVec + 1j*hcVec))]

    #q0 = M1/M2
    #q.append(M1/M2)
    #eta.append(M1*M2/(M*M))

    delta_t = delta_t_NR = np.abs(np.mean((np.diff(tVec))))

    fref = 0.075**(3/2) /MTSUN_SI/ PI/M
    fmin = 16.68 #0.045**(3/2) /MTSUN_SI/ PI/M

    hp, hc = get_td_waveform(approximant='EccentricTD', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin, eccentricity=e[Id])
    sp, sc = get_td_waveform(approximant='SEOBNRv4', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin)

    tshift = -(tshift_byM[hid])*M*MTSUN_SI   #minus only if data contains actual delta_tshift values

    #tmin = max(tVec[0],hp.sample_times[0],sp.sample_times[0]) + 0.04
    tmin1 = max(hp.sample_times[0]-tshift,sp.sample_times[0])

    #alternate
    plotband = np.where(tVec>=tmin1)
    hpVec = hpVec[plotband]
    hcVec = hcVec[plotband]
    tVec = tVec[plotband]


    h22Hyb = hpVec + 1j*hcVec
    phaseHyb = -np.unwrap(np.angle(h22Hyb)*2)/2
    phaseHyb = phaseHyb - phaseHyb[0]
    tVec = tVec - tVec[np.argmax(abs(h22Hyb))]

    #IMR circular
    #sp_intrp = interp1d(sp.sample_times, sp, kind='cubic')
    #sc_intrp = interp1d(sc.sample_times, sc, kind='cubic')
    #tImr_intrp=np.arange(tmin, sp.sample_times[-1], delta_t)
    #sp_intrp = sp_intrp(tImr_intrp)
    #sc_intrp = sc_intrp(tImr_intrp)
    #tImr = tImr_intrp
    #hpImr = sp_intrp
    #hcImr = sc_intrp
    
    plotImr = np.where(sp.sample_times>=tmin1)
    hpImr = sp[plotImr]
    hcImr = sc[plotImr]
    tImr = sp.sample_times[plotImr]
    
    #imrband = np.where(tImr<=tVec[-1])
    #tImr = tImr[imrband]
    #hpImr = hpImr[imrband]
    #hcImr = hcImr[imrband]
    h22Imr = hpImr + 1j*hcImr
    
    tshift = -(tshift_byM[hid])*M*MTSUN_SI #minus only if data contains actual delta_tshift values
    tmin = tImr[0]

    # interpolation EccentricTD
    hp_intrp = interp1d(hp.sample_times-tshift, hp, kind='cubic')#, fill_value='extrapolate')
    hc_intrp = interp1d(hc.sample_times-tshift, hc, kind='cubic')#, fill_value='extrapolate')
    # new (unifrom) sampling rate (for interpolation and further processing)
    tEcc_intrp=np.arange(tmin, hp.sample_times[-1]-tshift, delta_t)
    hp_intrp = hp_intrp(tEcc_intrp)
    hc_intrp = hc_intrp(tEcc_intrp)
    tEcc = tEcc_intrp
    hpEcc = hp_intrp
    hcEcc = hc_intrp
    h22Ecc = hpEcc + 1j*hcEcc
    #print(len(h22Ecc))
    phaseEcc = np.unwrap(np.angle(h22Ecc)*2)/2
    phaseImr = np.unwrap(np.angle(h22Imr)*2)/2

    #######
    dphase = phaseEcc[0] - phaseImr[0]
    #print(dphase)
    hp_new = real(h22Ecc * exp(-1j * dphase))
    hc_new = imag(h22Ecc * exp(-1j * dphase))

    phase_new = np.unwrap(np.angle(hp_new+1j*hc_new))
    #print(phase_new[0])

    phaseEcc = phase_new
    h22Ecc_new = hp_new+1j*hc_new
    #######



    "alternative model"

    Idxjoin = np.argmin(abs(tEcc-(tmatch_byM[hid]*M*MTSUN_SI)))
    idxstr = np.argmin(abs(tEcc-((tmatch_byM[hid]-500)*M*MTSUN_SI)))
    #idxstr = Idxjoin - 100
    amp = []
    count = 0
    length = Idxjoin - idxstr 
    for i in range(idxstr,Idxjoin):
        amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
        count = count+1

    t_model = np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))

    h22amp = np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
    h22amp_model = np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))

    omegaEcc = (M*MTSUN_SI/delta_t)*(np.gradient(phaseEcc))
    omegaImr = (M*MTSUN_SI/delta_t)*(np.gradient(phaseImr))


    fmatchIdx = []
    fmatch = []
    tfjoin = []
    print(tshift_byM[hid])
    ti = -6000
    tf = min(ti,ti-(-tshift_byM[hid])) #minus only if data contains actual delta_tshift values
    while(tf<-50):
        #print(tf)
        if tf>=tEcc[-1]/(M*MTSUN_SI)-50:
            break
        fjoin = np.argmin(abs(tEcc-tf*M*MTSUN_SI))
        tstop = min(tEcc[-1],-30*M*MTSUN_SI)
        lst = np.argmin(abs(tEcc-tstop))
        #lst = len(tEcc)
        indx = lst - fjoin

        a0 = []
        n = indx -1
        k = 0
        for i in range(fjoin,fjoin+indx):
            a0.append(((n-k)*omegaEcc[i]+k*omegaImr[i])/n)
            k = k+1
        #print(n)
        f1 = np.concatenate((omegaEcc[0:fjoin],a0))
        frequency_model = np.concatenate((f1,omegaImr[fjoin+indx:len(omegaImr)]))
        
        phase_f_model = np.cumsum(frequency_model)/(M*MTSUN_SI/delta_t)
    
        hp_f_model = h22amp_model * np.cos(phase_f_model)
        hp_temp_f = TimeSeries(hp_f_model,delta_t)
        htarg = TimeSeries(h22Hyb,delta_t)
        hp_targ_f = htarg.real()

        h_temp_f = hp_temp_f
        h_targ_f = hp_targ_f

        tlen=max(len(h_temp_f),len(h_targ_f))
        h_temp_f.resize(tlen)
        h_targ_f.resize(tlen)
        # Resize the waveforms to the same length
        #tlen = max(len(sp), len(hp))
        #sp.resize(tlen)
        #hp.resize(tlen)
        # Generate the aLIGO ZDHP PSD
        delta_f = 1.0 / h_targ_f.duration
        flen = tlen//2 + 1
        #flen = len(h_model)
        f_low = fmin
        #delta_f = h_model.delta_f
        psd = aLIGOZeroDetHighPower(flen, delta_f, f_low)
        # Note: This takes a while the first time as an FFT plan is generated
        # subsequent calls are much faster.
        m, i0 = match(h_targ_f, h_temp_f, psd=psd, low_frequency_cutoff=f_low)

        fmatch.append(m)
        fmatchIdx.append(fjoin)
        tfjoin.append(tEcc[fjoin])

        tf = tf + 1

    maxidx_f = np.argmax(fmatch)
    freqJoin.append(tfjoin[maxidx_f])
    freqJoin_byM.append(tfjoin[maxidx_f]/(M*MTSUN_SI))
    freqm.append(fmatch[maxidx_f])
    freqidx.append(fmatchIdx[maxidx_f])

344.99999999999994


In [6]:
print(freqm)
print(freqJoin_byM)

[0.6285521830250566]
[-51.166229039856184]


In [ ]:
[0.9760918054510879]
[-1388.8853362038024]